In [5]:
import pandas as pd
import aiohttp
from jikanpy import AioJikan
from time import sleep, time

In [3]:
anime_dataset = pd.read_csv('C:/Users/ASDW/Python/Projects/Recosys 2.0/datasets/anime_dataset/anime.csv')

In [4]:
ids = anime_dataset.MAL_ID.tolist()

In [5]:
jikan = jp.Jikan()

In [6]:
def get_cleared_recs(uncleared_recs):
    titles = []
    mal_ids = []
    i = 0
    while i != 7:
        try:
            title = uncleared_recs['recommendations'][i]['title']
            mal_id = uncleared_recs['recommendations'][i]['mal_id']
            titles.append(title)
            mal_ids.append(mal_id)
            i += 1
        except IndexError:
            titles.append('No')
            mal_ids.append('No')
            i += 1
    return titles, mal_ids

In [7]:
def get_recs(id):
    uncleared_recs = jikan.anime(id, extension='recommendations')
    recs = get_cleared_recs(uncleared_recs)
    return recs

In [8]:
anime_recs = pd.DataFrame(columns=['ID', 'Reco_Title', 'Reco_ID'])

In [13]:
start_time = time()
i = 0
not_responsed_ids = []
for item_id in ids[100:200]:
    try:
        if item_id not in anime_recs.ID.tolist():
            titles, mal_ids = get_recs(item_id)
            for i in range(7):
                anime_recs.loc[len(anime_recs.index)] = [item_id, titles[i], mal_ids[i]]
            if len(anime_recs) % 100 == 0:
                print(len(anime_recs))
            sleep(2)
        if item_id in anime_recs.ID.tolist():
            continue
    except jp.APIException:
        not_responsed_ids.append(item_id)
        sleep(60)
print("--- %s seconds ---" % (time() - start_time))

--- 1004.9282245635986 seconds ---


In [33]:
async def get_recs(ids):
    async with aiohttp.ClientSession() as session:
        all_recs = pd.DataFrame(columns=['ID', 'Reco_Title', 'Reco_ID'])
        for i in ids:
            body = {}
            async with session.get('https://api.jikan.moe/v3/anime/' + str(i) + '/recommendations/') as resp:
                anime = await resp.json()
                # на тот случай, если аниме под id равному i несуществует
                if 'recommendations' in anime.keys():
                    # на тот случай, если у нас меньше чем 7 рекомендаций
                    for j in range(len(anime['recommendations'][:7])):
                        all_recs.loc[len(all_recs.index)] = [i, anime['recommendations'][j]['title'], anime['recommendations'][j]['mal_id']]
        return all_recs

Если сравнивать так, то, действительно, разница есть и она большая.

In [32]:
start_time = time()
anime_recs = await get_recs(100, 200)
print('--- %s seconds ---' % (time() - start_time))

--- 773.4352695941925 seconds ---


In [37]:
anime_recs

,ID,Reco_Title,Reco_ID
0,100,Bishoujo Senshi Sailor Moon,530
1,100,Princess Tutu,721
2,100,Fushigi Yuugi,123
3,100,Hiiro no Kakera,12461
4,100,Neo Angelique Abyss,3615
...,...,...,...
531,199,Bakemono no Ko,28805
532,199,Hotarubi no Mori e,10408
533,199,Hoshi wo Ou Kodomo,9760
534,199,Majo no Takkyuubin,512


In [38]:
1799/7

257.0